## 1. Introducción

Este documento presenta un sistema completo de fine-tuning de modelos de lenguaje grandes utilizando la técnica LoRA (Low-Rank Adaptation). El sistema especializa el modelo DeepSeek-R1-Distill-Llama-8B para actuar como tutor de programación en TypeScript y conceptos de desarrollo.

### 1.1 Objetivos del Proyecto

- Adaptar un modelo de lenguaje base para respuestas específicas de dominio
- Implementar fine-tuning eficiente con mínimo uso de recursos
- Crear un tutor interactivo de programación
- Optimizar el modelo para despliegue local

### 1.2 Arquitectura del Sistema

El proyecto consta de cuatro componentes principales:

```
Sistema de Fine-Tuning
├── main.py          → Entrenamiento con LoRA
├── inference.py     → Sistema de inferencia interactivo
├── convert.py       → Conversión de modelos
└── Modelfile        → Configuración para Ollama
```

## 2. Técnica LoRA: Low-Rank Adaptation

### 2.1 Concepto Fundamental

LoRA es una técnica de fine-tuning que reduce drásticamente los recursos necesarios para adaptar modelos grandes:

**Problema del Fine-Tuning Tradicional:**
- Requiere actualizar millones o miles de millones de parámetros
- Necesita gran cantidad de memoria GPU
- Proceso lento y costoso
- Difícil de almacenar múltiples versiones especializadas

**Solución con LoRA:**
- Congela los pesos originales del modelo
- Añade matrices de bajo rango entrenables
- Actualiza solo un pequeño porcentaje de parámetros
- Almacena adaptadores ligeros (pocos MB vs. varios GB)

### 2.2 Parámetros LoRA

| Parámetro | Valor | Descripción |
|-----------|-------|-------------|
| `r` | 8 | Rango de las matrices de bajo rango |
| `lora_alpha` | 16 | Factor de escala para la actualización |
| `lora_dropout` | 0.1 | Dropout aplicado a capas LoRA |
| `target_modules` | q_proj, k_proj, v_proj, o_proj | Capas de atención modificadas |
| `bias` | none | No entrena sesgos adicionales |

### 2.3 Ventajas Técnicas

**Eficiencia de Memoria:**
- Modelo base: 8B parámetros congelados
- Adaptadores LoRA: ~5-10M parámetros entrenables
- Reducción: 99.9% menos parámetros a entrenar

**Modularidad:**
- Un modelo base + múltiples adaptadores
- Intercambio rápido entre especializaciones
- Fácil distribución y versionado

## 3. Modelo Base: DeepSeek-R1-Distill-Llama-8B

### 3.1 Características del Modelo

**Arquitectura:**
- Familia: Llama (Meta AI)
- Tamaño: 8 mil millones de parámetros
- Tipo: Modelo causal de lenguaje (autoregresivo)
- Contexto: Ventana de contexto extendida

**Especialización:**
- Destilado de DeepSeek-R1
- Optimizado para razonamiento
- Balance entre rendimiento y tamaño

### 3.2 Cuantización 4-bit

El sistema utiliza cuantización para reducir requisitos de hardware:

**Técnica BitsAndBytes:**
```python
load_in_4bit=True
bnb_4bit_quant_type="nf4"        # NormalFloat4
bnb_4bit_compute_dtype=float16   # Cálculos en FP16
bnb_4bit_use_double_quant=True   # Doble cuantización
```

**Impacto:**
- Memoria: ~5-6 GB VRAM vs. ~16 GB sin cuantización
- Velocidad: Mínima degradación con GPUs modernas
- Precisión: Pérdida menor al 1% en métricas estándar

## 4. Preparación del Dataset

### 4.1 Estructura de Datos

El dataset utiliza formato JSONL (JSON Lines) con estructura instrucción-respuesta:

```json
{
  "instruction": "Pregunta o tarea del usuario",
  "response": "Respuesta esperada del modelo"
}
```

### 4.2 Dominio del Dataset

El dataset contiene 278 pares de instrucción-respuesta sobre:

**Temas Principales:**
- Sistema de tipos de TypeScript
- Interfaces y tipos personalizados
- Enums y union types
- Genéricos y utility types
- Corrección de errores de tipo
- Patrones avanzados de TypeScript

### 4.3 Formato de Prompt

Cada ejemplo se formatea con un template consistente:

```python
prompt = f"Instrucción: {instruction}\nRespuesta:"
texto_completo = prompt + response
```

**Beneficios:**
- Estructura predecible para el modelo
- Fácil parsing durante inferencia
- Consistencia en formato de entrada/salida

### 4.4 Tokenización

El proceso de tokenización incluye:

- Conversión de texto a tokens numéricos
- Truncamiento a longitud máxima
- Padding token configurado como EOS
- Preservación del formato instrucción-respuesta

## 5. Proceso de Entrenamiento

### 5.1 Configuración de Hiperparámetros

| Parámetro | Valor | Justificación |
|-----------|-------|---------------|
| `per_device_train_batch_size` | 1 | Limita uso de memoria con modelo 8B |
| `gradient_accumulation_steps` | 10 | Simula batch size de 10 |
| `num_train_epochs` | 6 | Balance entre ajuste y sobreajuste |
| `fp16` | True | Entrenamiento en precisión mixta |
| `logging_steps` | 50 | Frecuencia de reporte de métricas |
| `save_steps` | 500 | Checkpoints periódicos |

### 5.2 Acumulación de Gradientes

Técnica para simular batches grandes con memoria limitada:

**Funcionamiento:**
1. Procesa 1 ejemplo (forward pass)
2. Calcula gradientes (backward pass)
3. Acumula gradientes sin actualizar pesos
4. Repite 10 veces
5. Actualiza pesos con gradiente acumulado

**Ventaja:** Batch efectivo de 10 con memoria de 1

### 5.3 Precision Mixta FP16

Combina diferentes precisiones numéricas:

- **FP16**: Cálculos rápidos en GPU
- **FP32**: Acumuladores para estabilidad

**Beneficios:**
- 2x más rápido en GPUs modernas
- 50% menos uso de memoria
- Mínima pérdida de precisión

### 5.4 Objetivo de Entrenamiento

El modelo se entrena con **Causal Language Modeling**:

- Predice el siguiente token dada la secuencia previa
- Función de pérdida: Cross-entropy sobre vocabulario
- Enmascaramiento: Solo calcula pérdida en tokens de respuesta

### 5.5 Monitoreo del Entrenamiento

Durante el entrenamiento se observa:

- **Loss (pérdida)**: Debe disminuir consistentemente
- **GPU utilization**: Verificar uso eficiente
- **Memory**: Confirmar que no hay OOM (Out Of Memory)
- **Steps/second**: Velocidad de entrenamiento

## 6. Sistema de Inferencia

### 6.1 Carga del Modelo

El sistema carga el modelo en tres pasos:

```python
# 1. Modelo base cuantizado
base_model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    load_in_4bit=True
)

# 2. Adaptadores LoRA
model = PeftModel.from_pretrained(base_model, "./lora-tutor")

# 3. Tokenizer
tokenizer = AutoTokenizer.from_pretrained(...)
```

### 6.2 Parámetros de Generación

| Parámetro | Valor | Efecto |
|-----------|-------|--------|
| `max_new_tokens` | 515 | Longitud máxima de respuesta |
| `temperature` | 0.003 | Generación casi determinista |
| `do_sample` | True | Habilita muestreo probabilístico |
| `top_p` | 0.85 | Nucleus sampling (Top-P) |
| `repetition_penalty` | 1.15 | Penaliza repeticiones |

### 6.3 Estrategias de Decodificación

#### Temperature

Controla la aleatoriedad de las predicciones:

- **0.003**: Casi greedy, muy determinista
- Selecciona tokens con mayor probabilidad
- Ideal para respuestas técnicas precisas

#### Nucleus Sampling (Top-P)

Selecciona del conjunto más probable de tokens:

- **top_p = 0.85**: Considera tokens que suman 85% probabilidad
- Elimina opciones muy improbables
- Balance entre coherencia y diversidad

#### Repetition Penalty

Penaliza tokens ya generados:

- **1.15**: Penalización moderada
- Previene bucles de repetición
- Mantiene variedad léxica

### 6.4 Flujo de Inferencia

Proceso completo de generación:

1. **Entrada del usuario**: Pregunta en lenguaje natural
2. **Formateo**: Añadir template de instrucción
3. **Tokenización**: Convertir a IDs numéricos
4. **Generación**: Producir tokens autorregresvamente
5. **Decodificación**: Convertir IDs a texto
6. **Post-procesamiento**: Extraer solo la respuesta

## 7. Conversión y Despliegue

### 7.1 Fusión de Adaptadores

El script `convert.py` fusiona LoRA con el modelo base:

**Proceso:**
```python
# Cargar modelo base + LoRA
model = AutoModelForCausalLM.from_pretrained(base_model)
model = PeftModel.from_pretrained(model, lora_path)

# Fusionar
merged_model = model.merge_and_unload()

# Guardar
merged_model.save_pretrained(output_path)
```

**Resultado:**
- Modelo unificado sin dependencias de PEFT
- Más rápido en inferencia
- Compatible con herramientas estándar

### 7.2 Conversión a GGUF

GGUF es un formato optimizado para inferencia en CPU:

**Características:**
- Cuantización adicional (Q4, Q5, Q8)
- Optimizado para llama.cpp
- Ejecución eficiente en CPU
- Compatible con Ollama

**Limitación:**
- Puede fallar con modelos ya cuantizados en 4-bit
- Requiere modelo en FP16/FP32 para conversión óptima

### 7.3 Integración con Ollama

El `Modelfile` define la configuración para Ollama:

```
FROM deepseek-ai/DeepSeek-R1-Distill-Llama-8B
ADAPTER ./lora-tutor
SYSTEM "Prompt de sistema personalizado"
```

**Ventajas de Ollama:**
- API REST simple
- Gestión de modelos locales
- Compatible con múltiples frameworks
- Interfaz web incluida

## 8. Optimizaciones de Memoria

### 8.1 Cuantización en Cascada

El sistema aplica cuantización en múltiples niveles:

**Nivel 1: Modelo Base (4-bit NF4)**
- Reduce 8B parámetros de FP16 a 4-bit
- Ahorro: ~75% de memoria
- De ~16 GB a ~4 GB

**Nivel 2: Doble Cuantización**
- Cuantiza los factores de escala
- Ahorro adicional: ~10-15%
- Mínimo impacto en calidad

### 8.2 Gestión de Dispositivos

Estrategias para distribución de memoria:

```python
# Automática
device_map="auto"  # Distribuye capas óptimamente

# Manual
max_memory={0: "6GB"}  # Limita uso de GPU 0
```

**Offloading:**
- Capas menos usadas a CPU/Disco
- Mantiene capas críticas en GPU
- Trade-off memoria vs. velocidad

### 8.3 Configuración de VRAM

Requisitos según configuración:

| Configuración | VRAM | Velocidad |
|---------------|------|----------|
| 4-bit + LoRA | 5-6 GB | Rápida |
| 8-bit | 8-10 GB | Media |
| FP16 | 16+ GB | Muy rápida |
| CPU Offload | 2-3 GB | Lenta |

## 9. Evaluación del Modelo

### 9.1 Métricas de Calidad

**Durante Entrenamiento:**
- Loss de entrenamiento: Indicador de ajuste
- Perplexity: Medida de incertidumbre del modelo
- Convergencia: Estabilización de la pérdida

**Durante Inferencia:**
- Relevancia de respuestas
- Precisión técnica
- Coherencia en formato
- Ausencia de alucinaciones

### 9.2 Evaluación Cualitativa

Aspectos a verificar manualmente:

**Conocimiento del Dominio:**
- Corrección de conceptos de TypeScript
- Sintaxis apropiada en ejemplos
- Terminología técnica precisa

**Capacidad Pedagógica:**
- Explicaciones claras y concisas
- Ejemplos prácticos relevantes
- Estructura de respuesta didáctica

**Consistencia:**
- Mismo formato en todas las respuestas
- Tono profesional uniforme
- Adherencia al template

### 9.3 Casos de Prueba

Categorías de preguntas para validación:

1. **Definiciones**: "¿Qué es una interfaz?"
2. **Ejemplos**: "Muestra un ejemplo de genéricos"
3. **Correcciones**: "Corrige este error de tipo"
4. **Comparaciones**: "Diferencia entre type y interface"
5. **Aplicaciones**: "¿Cuándo usar enums?"

## 10. Arquitectura del Sistema

### 10.1 Componentes y Flujos

```
┌─────────────────────────────────────────────────┐
│          FASE DE ENTRENAMIENTO                  │
├─────────────────────────────────────────────────┤
│                                                 │
│  Dataset (JSONL) → Tokenización → Batching    │
│         ↓                                       │
│  Modelo Base (8B) + Cuantización 4-bit        │
│         ↓                                       │
│  Configuración LoRA (r=8, alpha=16)           │
│         ↓                                       │
│  Entrenamiento (6 épocas, batch_size=10)      │
│         ↓                                       │
│  Guardado de Adaptadores (~10 MB)             │
│                                                 │
└─────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────┐
│          FASE DE INFERENCIA                     │
├─────────────────────────────────────────────────┤
│                                                 │
│  Pregunta Usuario → Formateo Template         │
│         ↓                                       │
│  Modelo Base + Adaptadores LoRA               │
│         ↓                                       │
│  Generación (temp=0.003, top_p=0.85)          │
│         ↓                                       │
│  Post-procesamiento y Visualización           │
│                                                 │
└─────────────────────────────────────────────────┘
```

### 10.2 Estructura de Archivos

```
fine/
├── main.py                    # Script de entrenamiento
├── inference.py               # Sistema interactivo
├── convert.py                 # Fusión y conversión
├── Modelfile                  # Config Ollama
├── respuestas_fixed.jsonl     # Dataset de entrenamiento
└── lora-tutor/                # Adaptadores entrenados
    ├── adapter_config.json    # Configuración LoRA
    ├── adapter_model.safetensors  # Pesos LoRA
    └── README.md              # Metadatos del modelo
```

### 10.3 Dependencias Principales

```python
transformers>=4.30.0    # Framework de modelos
peft>=0.4.0            # Parameter-Efficient Fine-Tuning
bitsandbytes>=0.40.0   # Cuantización
accelerate>=0.20.0     # Optimizaciones de hardware
datasets>=2.12.0       # Manejo de datos
torch>=2.0.0           # Framework de deep learning
```